In [1]:
import polars as pl
import xarray as xr

from datetime import timedelta

In [2]:
path = r"C:\python_project\energy_power_pred\data\raw_data\train\nwp_data_train\1\NWP_1\20240101.nc"
nc_data = xr.load_dataset(path)
nc_data

<xarray.Dataset> Size: 186kB
Dimensions:    (time: 1, channel: 8, lead_time: 24, lat: 11, lon: 11)
Coordinates:
  * time       (time) datetime64[ns] 8B 2024-01-01T16:00:00
  * channel    (channel) <U4 128B 'ghi' 'poai' 'sp' 't2m' ... 'tp' 'u100' 'v100'
  * lat        (lat) int64 88B 0 1 2 3 4 5 6 7 8 9 10
  * lon        (lon) int64 88B 0 1 2 3 4 5 6 7 8 9 10
  * lead_time  (lead_time) int64 192B 0 1 2 3 4 5 6 7 ... 17 18 19 20 21 22 23
Data variables:
    data       (time, lead_time, channel, lat, lon) float64 186kB 0.0 ... -4.329

In [3]:
nc_data.to_dataframe() # 好了，这下看懂一点儿了，pandas 嘛, 多重索引

data
time                channel lead_time lat lon          
2024-01-01 16:00:00 ghi     0         0   0    0.000000
                                          1    0.000000
                                          2    0.000000
                                          3    0.000000
                                          4    0.000000
...                                                 ...
                    v100    23        10  6   -3.089172
                                          7   -3.175110
                                          8   -3.438782
                                          9   -3.884094
                                          10  -4.329407

[23232 rows x 1 columns]

In [4]:
11*11*24*8

23232

In [5]:
(
    nc_data
    .to_dataframe()
    .reset_index()
)

time channel  lead_time  lat  lon      data
0     2024-01-01 16:00:00     ghi          0    0    0  0.000000
1     2024-01-01 16:00:00     ghi          0    0    1  0.000000
2     2024-01-01 16:00:00     ghi          0    0    2  0.000000
3     2024-01-01 16:00:00     ghi          0    0    3  0.000000
4     2024-01-01 16:00:00     ghi          0    0    4  0.000000
...                   ...     ...        ...  ...  ...       ...
23227 2024-01-01 16:00:00    v100         23   10    6 -3.089172
23228 2024-01-01 16:00:00    v100         23   10    7 -3.175110
23229 2024-01-01 16:00:00    v100         23   10    8 -3.438782
23230 2024-01-01 16:00:00    v100         23   10    9 -3.884094
23231 2024-01-01 16:00:00    v100         23   10   10 -4.329407

[23232 rows x 6 columns]

In [6]:
data_pl = pl.DataFrame(
    nc_data
    .to_dataframe()
    .reset_index()
)
data_pl

shape: (23_232, 6)
┌─────────────────────┬─────────┬───────────┬─────┬─────┬───────────┐
│ time                ┆ channel ┆ lead_time ┆ lat ┆ lon ┆ data      │
│ ---                 ┆ ---     ┆ ---       ┆ --- ┆ --- ┆ ---       │
│ datetime[ns]        ┆ str     ┆ i64       ┆ i64 ┆ i64 ┆ f64       │
╞═════════════════════╪═════════╪═══════════╪═════╪═════╪═══════════╡
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 0   ┆ 0.0       │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 1   ┆ 0.0       │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 2   ┆ 0.0       │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 3   ┆ 0.0       │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 4   ┆ 0.0       │
│ …                   ┆ …       ┆ …         ┆ …   ┆ …   ┆ …         │
│ 2024-01-01 16:00:00 ┆ v100    ┆ 23        ┆ 10  ┆ 6   ┆ -3.089172 │
│ 2024-01-01 16:00:00 ┆ v100    ┆ 23        ┆ 10  ┆ 7   ┆ -3.17511  │
│ 2024-01-01 16:00:00 ┆ v100    ┆ 23        ┆ 10  ┆ 8   ┆ -3.438782 │
│ 2024-01-01 16:00:00 ┆ v100    ┆ 23        ┆ 10  ┆ 9   ┆ -3.884094 │
│ 2024-01-01 16:00:00 ┆ v100    ┆ 23        ┆ 10  ┆ 10  ┆ -4.329407 │
└─────────────────────┴─────────┴───────────┴─────┴─────┴───────────┘

In [7]:
data_pl.null_count()

shape: (1, 6)
┌──────┬─────────┬───────────┬─────┬─────┬──────┐
│ time ┆ channel ┆ lead_time ┆ lat ┆ lon ┆ data │
│ ---  ┆ ---     ┆ ---       ┆ --- ┆ --- ┆ ---  │
│ u32  ┆ u32     ┆ u32       ┆ u32 ┆ u32 ┆ u32  │
╞══════╪═════════╪═══════════╪═════╪═════╪══════╡
│ 0    ┆ 0       ┆ 0         ┆ 0   ┆ 0   ┆ 0    │
└──────┴─────────┴───────────┴─────┴─────┴──────┘

In [8]:
for i in ["time", "channel", "lead_time", "lat", "lon"]:
    print(f"{i}: {data_pl[i].unique().to_numpy().squeeze()}")

time: 2024-01-01T16:00:00.000000000
channel: ['sp' 't2m' 'u100' 'v100' 'tp' 'tcc' 'poai' 'ghi']
lead_time: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
lat: [ 0  1  2  3  4  5  6  7  8  9 10]
lon: [ 0  1  2  3  4  5  6  7  8  9 10]


In [9]:
from datetime import timedelta

data_pl_bjtime = (
    data_pl
    .with_columns([
        # 将国际时间加 8 小时转换成北京时间
        (pl.col("time") + timedelta(hours=8)).alias("release_time")
    ])
    .with_columns([
        # 计算天气预报时间，这个时间是后一天的天气预测时间，
        # 即在世界时1.1日发布的北京时间1.2日的天气预测数据
        # 我么要做的是根据1.2日的天气预报数据预测1.2的发电功率
        (
            pl.col("release_time") + 
            pl.col("lead_time").cast(pl.Int32) * timedelta(hours=1)
        )
        .alias("forecast_time")
    ])
    .drop(["time", "lead_time"])
    .select(['release_time', 'forecast_time', 'channel', 'lat', 'lon', 'data'])
    
)
data_pl_bjtime

shape: (23_232, 6)
┌─────────────────────┬─────────────────────┬─────────┬─────┬─────┬───────────┐
│ release_time        ┆ forecast_time       ┆ channel ┆ lat ┆ lon ┆ data      │
│ ---                 ┆ ---                 ┆ ---     ┆ --- ┆ --- ┆ ---       │
│ datetime[μs]        ┆ datetime[μs]        ┆ str     ┆ i64 ┆ i64 ┆ f64       │
╞═════════════════════╪═════════════════════╪═════════╪═════╪═════╪═══════════╡
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ ghi     ┆ 0   ┆ 0   ┆ 0.0       │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ ghi     ┆ 0   ┆ 1   ┆ 0.0       │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ ghi     ┆ 0   ┆ 2   ┆ 0.0       │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ ghi     ┆ 0   ┆ 3   ┆ 0.0       │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ ghi     ┆ 0   ┆ 4   ┆ 0.0       │
│ …                   ┆ …                   ┆ …       ┆ …   ┆ …   ┆ …         │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ v100    ┆ 10  ┆ 6   ┆ -3.089172 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ v100    ┆ 10  ┆ 7   ┆ -3.17511  │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ v100    ┆ 10  ┆ 8   ┆ -3.438782 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ v100    ┆ 10  ┆ 9   ┆ -3.884094 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ v100    ┆ 10  ┆ 10  ┆ -4.329407 │
└─────────────────────┴─────────────────────┴─────────┴─────┴─────┴───────────┘

In [10]:
# 提取一个变量
(
    data_pl_bjtime
    .filter(
        pl.col("channel") == "tcc"
    )
)

shape: (2_904, 6)
┌─────────────────────┬─────────────────────┬─────────┬─────┬─────┬──────────┐
│ release_time        ┆ forecast_time       ┆ channel ┆ lat ┆ lon ┆ data     │
│ ---                 ┆ ---                 ┆ ---     ┆ --- ┆ --- ┆ ---      │
│ datetime[μs]        ┆ datetime[μs]        ┆ str     ┆ i64 ┆ i64 ┆ f64      │
╞═════════════════════╪═════════════════════╪═════════╪═════╪═════╪══════════╡
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ tcc     ┆ 0   ┆ 0   ┆ 1.0      │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ tcc     ┆ 0   ┆ 1   ┆ 0.974976 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ tcc     ┆ 0   ┆ 2   ┆ 0.862152 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ tcc     ┆ 0   ┆ 3   ┆ 0.777405 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ tcc     ┆ 0   ┆ 4   ┆ 0.862396 │
│ …                   ┆ …                   ┆ …       ┆ …   ┆ …   ┆ …        │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ tcc     ┆ 10  ┆ 6   ┆ 0.050415 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ tcc     ┆ 10  ┆ 7   ┆ 0.05545  │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ tcc     ┆ 10  ┆ 8   ┆ 0.070038 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ tcc     ┆ 10  ┆ 9   ┆ 0.089325 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ tcc     ┆ 10  ┆ 10  ┆ 0.092407 │
└─────────────────────┴─────────────────────┴─────────┴─────┴─────┴──────────┘

In [11]:
# 上面数据长度也是2904, 正好是一天的数据
11*11*24

2904

In [12]:
(
    data_pl_bjtime
    .filter(
        pl.col("channel") == "tcc"
    )
    .rename({"data": "tcc"})
    .drop("channel")
)

shape: (2_904, 5)
┌─────────────────────┬─────────────────────┬─────┬─────┬──────────┐
│ release_time        ┆ forecast_time       ┆ lat ┆ lon ┆ tcc      │
│ ---                 ┆ ---                 ┆ --- ┆ --- ┆ ---      │
│ datetime[μs]        ┆ datetime[μs]        ┆ i64 ┆ i64 ┆ f64      │
╞═════════════════════╪═════════════════════╪═════╪═════╪══════════╡
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 0   ┆ 1.0      │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 1   ┆ 0.974976 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 2   ┆ 0.862152 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 3   ┆ 0.777405 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 4   ┆ 0.862396 │
│ …                   ┆ …                   ┆ …   ┆ …   ┆ …        │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 6   ┆ 0.050415 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 7   ┆ 0.05545  │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 8   ┆ 0.070038 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 9   ┆ 0.089325 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 10  ┆ 0.092407 │
└─────────────────────┴─────────────────────┴─────┴─────┴──────────┘

In [13]:
tcc_data_pl = (
    data_pl_bjtime
    .filter(
        pl.col("channel") == "tcc"
    )
    .rename({"data": "tcc"})
    .drop("channel")
)
t2m_data_pl = (
    data_pl_bjtime
    .filter(
        pl.col("channel") == "t2m"
    )
    .rename({"data": "t2m"})
    .drop("channel")
)

(
    tcc_data_pl
    .join(
        t2m_data_pl,
        on=["release_time", "forecast_time", "lat", "lon"],
        how="left"
    )
)

shape: (2_904, 6)
┌─────────────────────┬─────────────────────┬─────┬─────┬──────────┬────────────┐
│ release_time        ┆ forecast_time       ┆ lat ┆ lon ┆ tcc      ┆ t2m        │
│ ---                 ┆ ---                 ┆ --- ┆ --- ┆ ---      ┆ ---        │
│ datetime[μs]        ┆ datetime[μs]        ┆ i64 ┆ i64 ┆ f64      ┆ f64        │
╞═════════════════════╪═════════════════════╪═════╪═════╪══════════╪════════════╡
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 0   ┆ 1.0      ┆ 288.285339 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 1   ┆ 0.974976 ┆ 289.039246 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 2   ┆ 0.862152 ┆ 290.203308 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 3   ┆ 0.777405 ┆ 290.490417 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 4   ┆ 0.862396 ┆ 290.019714 │
│ …                   ┆ …                   ┆ …   ┆ …   ┆ …        ┆ …          │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 6   ┆ 0.050415 ┆ 293.637787 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 7   ┆ 0.05545  ┆ 293.637787 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 8   ┆ 0.070038 ┆ 293.610443 │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 9   ┆ 0.089325 ┆ 293.63974  │
│ 2024-01-02 00:00:00 ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 10  ┆ 0.092407 ┆ 293.622162 │
└─────────────────────┴─────────────────────┴─────┴─────┴──────────┴────────────┘

In [14]:
# 我们把其他的变量数据也都提取出来，这里需要用到 group_by
# 先简单看看怎么 group_by
for i in data_pl.group_by("channel", maintain_order=True):
    print(i)
    break

(('ghi',), shape: (2_904, 6)
┌─────────────────────┬─────────┬───────────┬─────┬─────┬──────┐
│ time                ┆ channel ┆ lead_time ┆ lat ┆ lon ┆ data │
│ ---                 ┆ ---     ┆ ---       ┆ --- ┆ --- ┆ ---  │
│ datetime[ns]        ┆ str     ┆ i64       ┆ i64 ┆ i64 ┆ f64  │
╞═════════════════════╪═════════╪═══════════╪═════╪═════╪══════╡
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 0   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 1   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 2   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 3   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 4   ┆ 0.0  │
│ …                   ┆ …       ┆ …         ┆ …   ┆ …   ┆ …    │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 6   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 7   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 8   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 9   ┆ 0.0

In [15]:
for (channel_i, ), per_channel_data in data_pl.group_by("channel", maintain_order=True):
    print(channel_i, "\n")    # 可以看出来 channel 就是变量名
    print(per_channel_data)
    break

ghi 

shape: (2_904, 6)
┌─────────────────────┬─────────┬───────────┬─────┬─────┬──────┐
│ time                ┆ channel ┆ lead_time ┆ lat ┆ lon ┆ data │
│ ---                 ┆ ---     ┆ ---       ┆ --- ┆ --- ┆ ---  │
│ datetime[ns]        ┆ str     ┆ i64       ┆ i64 ┆ i64 ┆ f64  │
╞═════════════════════╪═════════╪═══════════╪═════╪═════╪══════╡
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 0   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 1   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 2   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 3   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 0         ┆ 0   ┆ 4   ┆ 0.0  │
│ …                   ┆ …       ┆ …         ┆ …   ┆ …   ┆ …    │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 6   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 7   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 8   ┆ 0.0  │
│ 2024-01-01 16:00:00 ┆ ghi     ┆ 23        ┆ 10  ┆ 9   ┆ 0.0  │
│

In [16]:
df_processed = None # df_processed 只存储一天的数据
# 循环处理每个变量（chaanel 这一列是八个变量）
for (channel_i, ), per_channel_data in data_pl.group_by("channel", 
                                                        maintain_order=True):
    # print(channel_i, "\n")
    per_channel_data = (
        per_channel_data
        .with_columns([
            (pl.col("time") + timedelta(hours=8)).alias("release_time")
        ])
        .with_columns([
            (pl.col("release_time") + pl.col("lead_time")
             .cast(pl.Int32) * timedelta(hours=1))
            .alias("forecast_time")
        ])
        .rename({"data": channel_i})
        .drop(["time", "channel", "lead_time"])
        .select(['release_time', 'forecast_time', 'lat', 'lon', channel_i])
        .sort(["forecast_time", "lat", "lon"])
    ) 
    
    if df_processed is None:
        df_processed = per_channel_data
    else:
        df_processed =(
            df_processed
            .join(
                per_channel_data, 
                on=["release_time", "forecast_time", "lat", "lon"], 
                how="left"
            )
            .sort(["forecast_time", "lat", "lon"])
            
        )
df_processed
    

shape: (2_904, 12)
┌──────────────┬─────────────────────┬─────┬─────┬───┬──────────┬──────────┬───────────┬───────────┐
│ release_time ┆ forecast_time       ┆ lat ┆ lon ┆ … ┆ tcc      ┆ tp       ┆ u100      ┆ v100      │
│ ---          ┆ ---                 ┆ --- ┆ --- ┆   ┆ ---      ┆ ---      ┆ ---       ┆ ---       │
│ datetime[μs] ┆ datetime[μs]        ┆ i64 ┆ i64 ┆   ┆ f64      ┆ f64      ┆ f64       ┆ f64       │
╞══════════════╪═════════════════════╪═════╪═════╪═══╪══════════╪══════════╪═══════════╪═══════════╡
│ 2024-01-02   ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 0   ┆ … ┆ 1.0      ┆ 0.0      ┆ -3.21907  ┆ 1.913437  │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 1   ┆ … ┆ 0.974976 ┆ 0.0      ┆ -1.407547 ┆ 1.079453  │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 2   ┆ … ┆ 0.862152 ┆ 0.0      ┆ -0.339188 ┆ 1.008163  │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 3   ┆ … ┆ 0.777405 ┆ 0.0      ┆ -0.652664 ┆ 0.768906  │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 00:00:00 ┆ 0   ┆ 4   ┆ … ┆ 0.862396 ┆ 0.0      ┆ -1.319656 ┆ -0.36879  │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ …            ┆ …                   ┆ …   ┆ …   ┆ … ┆ …        ┆ …        ┆ …         ┆ …         │
│ 2024-01-02   ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 6   ┆ … ┆ 0.050415 ┆ 0.000084 ┆ -6.908569 ┆ -3.089172 │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 7   ┆ … ┆ 0.05545  ┆ 0.000046 ┆ -6.976929 ┆ -3.17511  │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 8   ┆ … ┆ 0.070038 ┆ 0.000061 ┆ -6.937866 ┆ -3.438782 │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 9   ┆ … ┆ 0.089325 ┆ 0.000076 ┆ -6.707397 ┆ -3.884094 │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
│ 2024-01-02   ┆ 2024-01-02 23:00:00 ┆ 10  ┆ 10  ┆ … ┆ 0.092407 ┆ 0.000038 ┆ -6.330444 ┆ -4.329407 │
│ 00:00:00     ┆                     ┆     ┆     ┆   ┆          ┆          ┆           ┆           │
└──────────────┴─────────────────────┴─────┴─────┴───┴──────────┴──────────┴───────────┴───────────┘

In [17]:
# 查看数据没问题，到这基本就结束了
df_processed.null_count()

shape: (1, 12)
┌──────────────┬───────────────┬─────┬─────┬───┬─────┬─────┬──────┬──────┐
│ release_time ┆ forecast_time ┆ lat ┆ lon ┆ … ┆ tcc ┆ tp  ┆ u100 ┆ v100 │
│ ---          ┆ ---           ┆ --- ┆ --- ┆   ┆ --- ┆ --- ┆ ---  ┆ ---  │
│ u32          ┆ u32           ┆ u32 ┆ u32 ┆   ┆ u32 ┆ u32 ┆ u32  ┆ u32  │
╞══════════════╪═══════════════╪═════╪═════╪═══╪═════╪═════╪══════╪══════╡
│ 0            ┆ 0             ┆ 0   ┆ 0   ┆ … ┆ 0   ┆ 0   ┆ 0    ┆ 0    │
└──────────────┴───────────────┴─────┴─────┴───┴─────┴─────┴──────┴──────┘